# Authentication Endpoints Testing

This notebook allows you to test all authentication-related endpoints interactively.

## Endpoints Covered:
- `/api/auth/registration/` - User registration
- `/api/auth/login/` - User login
- `/api/auth/logout/` - User logout
- `/api/auth/user/` - Get current user details
- `/api/auth/token/refresh/` - Refresh JWT token
- `/api/auth/password/reset/` - Password reset request
- `/api/auth/password/reset/confirm/` - Confirm password reset
- `/api/auth/password/change/` - Change password (authenticated)

## Setup

Import required libraries and set base URL

In [4]:
import requests
import json
from pprint import pprint

# Base URL
BASE_URL = "http://localhost:8000"

# Store tokens globally
access_token = None
refresh_token = None

def print_response(response):
    """Pretty print response"""
    print(f"Status Code: {response.status_code}")
    print(f"\nResponse Headers:")
    pprint(dict(response.headers))
    print(f"\nResponse Body:")
    try:
        pprint(response.json())
    except:
        print(response.text)

print("✅ Setup complete!")

✅ Setup complete!


## 1. User Registration

Register a new user account (email-based, no username required)

In [5]:
# Registration data
registration_data = {
    "email": "testuser@example.com",
    "password1": "SecurePass123!",
    "password2": "SecurePass123!"
}

response = requests.post(
    f"{BASE_URL}/api/auth/registration/",
    json=registration_data
)

print_response(response)

# Store tokens if registration successful
if response.status_code == 201:
    data = response.json()
    access_token = data.get('access')
    refresh_token = data.get('refresh')
    print(f"\n✅ Registration successful!")
    print(f"Access token stored: {access_token[:50]}...")

Status Code: 500

Response Headers:
{'Content-Length': '223540',
 'Content-Type': 'text/html; charset=utf-8',
 'Cross-Origin-Opener-Policy': 'same-origin',
 'Date': 'Fri, 14 Nov 2025 07:14:51 GMT',
 'Referrer-Policy': 'same-origin',
 'Server': 'WSGIServer/0.2 CPython/3.11.14',
 'Vary': 'origin, Cookie',
 'X-Content-Type-Options': 'nosniff',
 'X-Frame-Options': 'DENY'}

Response Body:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta name="robots" content="NONE,NOARCHIVE">
  <title>IntegrityError
          at /api/auth/registration/</title>
  <style>
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font-family: sans-serif; background-color:#fff; color:#000; }
    body > :where(header, main, footer) { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; }
    h2 { margin-bottom:.8em; }
    h3 { margin:1em 0 .5em 0; }
    h4 { margin:0 0 .5em 0; font-weig

## 2. User Login

Login with email and password to get JWT tokens

In [ ]:
# Login data
login_data = {
    "email": "admin@replycompass.com",
    "password": "admin123"
}

response = requests.post(
    f"{BASE_URL}/api/auth/login/",
    json=login_data
)

print_response(response)

# Store tokens if login successful
if response.status_code == 200:
    data = response.json()
    access_token = data.get('access')
    refresh_token = data.get('refresh')
    print(f"\n✅ Login successful!")
    print(f"Access token: {access_token[:50]}...")
    print(f"Refresh token: {refresh_token[:50]}...")

## 3. Get Current User Details

Fetch authenticated user's profile information

In [ ]:
if not access_token:
    print("❌ No access token! Please login first.")
else:
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(
        f"{BASE_URL}/api/auth/user/",
        headers=headers
    )
    
    print_response(response)

## 4. Refresh Token

Get a new access token using refresh token

In [ ]:
if not refresh_token:
    print("❌ No refresh token! Please login first.")
else:
    response = requests.post(
        f"{BASE_URL}/api/auth/token/refresh/",
        json={"refresh": refresh_token}
    )
    
    print_response(response)
    
    if response.status_code == 200:
        data = response.json()
        access_token = data.get('access')
        print(f"\n✅ Token refreshed!")
        print(f"New access token: {access_token[:50]}...")

## 5. Change Password

Change password for authenticated user

In [ ]:
if not access_token:
    print("❌ No access token! Please login first.")
else:
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    change_password_data = {
        "old_password": "admin123",
        "new_password1": "NewSecurePass123!",
        "new_password2": "NewSecurePass123!"
    }
    
    response = requests.post(
        f"{BASE_URL}/api/auth/password/change/",
        json=change_password_data,
        headers=headers
    )
    
    print_response(response)

## 6. Password Reset Request

Request password reset email

In [ ]:
reset_data = {
    "email": "admin@replycompass.com"
}

response = requests.post(
    f"{BASE_URL}/api/auth/password/reset/",
    json=reset_data
)

print_response(response)

## 7. Logout

Logout and blacklist refresh token

In [ ]:
if not access_token:
    print("❌ No access token! Please login first.")
else:
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.post(
        f"{BASE_URL}/api/auth/logout/",
        headers=headers
    )
    
    print_response(response)
    
    if response.status_code == 200:
        print("\n✅ Logged out successfully!")
        access_token = None
        refresh_token = None

## 8. Test Invalid Scenarios

Test various error cases

In [ ]:
print("=" * 60)
print("Test 1: Login with wrong password")
print("=" * 60)

response = requests.post(
    f"{BASE_URL}/api/auth/login/",
    json={"email": "admin@replycompass.com", "password": "wrongpass"}
)
print_response(response)

print("\n" + "=" * 60)
print("Test 2: Registration with mismatched passwords")
print("=" * 60)

response = requests.post(
    f"{BASE_URL}/api/auth/registration/",
    json={
        "email": "test@example.com",
        "password1": "Pass123!",
        "password2": "Pass456!"
    }
)
print_response(response)

print("\n" + "=" * 60)
print("Test 3: Access protected endpoint without token")
print("=" * 60)

response = requests.get(f"{BASE_URL}/api/auth/user/")
print_response(response)